
# 1) Understanding LLM Input Data

Packages that are being used in this notebook:

In [29]:
from google.colab import files
uploaded = files.upload()
print(uploaded)

Saving supplementary.py to supplementary.py
{'supplementary.py': b'# Copyright (c) Sebastian Raschka under Apache License 2.0 (see LICENSE.txt).\n# Source for "Build a Large Language Model From Scratch"\n#   - https://www.manning.com/books/build-a-large-language-model-from-scratch\n# Code: https://github.com/rasbt/LLMs-from-scratch\n\nimport torch\nimport tiktoken\nfrom torch.utils.data import Dataset, DataLoader\n\n\nclass GPTDatasetV1(Dataset):\n    def __init__(self, txt, tokenizer, max_length, stride):\n        self.input_ids = []\n        self.target_ids = []\n\n        # Tokenize the entire text\n        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})\n\n        # Use a sliding window to chunk the book into overlapping sequences of max_length\n        for i in range(0, len(token_ids) - max_length, stride):\n            input_chunk = token_ids[i:i + max_length]\n            target_chunk = token_ids[i + 1: i + max_length + 1]\n            self.input_ids.append(

In [6]:
from importlib.metadata import version


print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.5.1+cu124
tiktoken version: 0.8.0


- This notebook provides a brief overview of the data preparation and sampling procedures to get input data "ready" for an LLM
- Understanding what the input data looks like is a great first step towards understanding how LLMs work

<img src="./figures/01.png" width="1000px">


# 2.1 Tokenizing text

- In this section, we tokenize text, which means breaking text into smaller units, such as individual words and punctuation characters

<img src="figures/02.png" width="800px">

- Load raw text we want to work with
- [The Verdict by Edith Wharton](https://en.wikisource.org/wiki/The_Verdict) is a public domain short story

In [2]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- The goal is to tokenize and embed this text for an LLM
- Let's develop a simple tokenizer based on some simple sample text that we can then later apply to the text above

<img src="figures/03.png" width="690px">

- The following regular expression will split on whitespaces and punctuation

We have one word per token and we also have the whitespaces

In [8]:
import re

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
print(preprocessed[:38]) #first 38 tokens

['I', ' ', 'HAD', ' ', 'always', ' ', 'thought', ' ', 'Jack', ' ', 'Gisburn', ' ', 'rather', ' ', 'a', ' ', 'cheap', ' ', 'genius', '--', 'though', ' ', 'a', ' ', 'good', ' ', 'fellow', ' ', 'enough', '--', 'so', ' ', 'it', ' ', 'was', ' ', 'no', ' ']


In [9]:
print("Number of tokens:", len(preprocessed))

Number of tokens: 9235


In [10]:
print("Number of unique tokens:", len(set(preprocessed)))

Number of unique tokens: 1133



# 2.2 Converting tokens into token IDs

- Next, we convert the text tokens into token IDs that we can process via embedding layers later
- For this we first need to build a vocabulary

<img src="figures/04.png" width="900px">

- The vocabulary contains the unique words in the input text

In [12]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1133


We are mapping the tokens into IDs.

In [14]:
vocab = {token:integer for integer,token in enumerate(all_words)}

- Below are the first 50 entries in this vocabulary:
- The punctuation comes first and then we have them in alphabetical order

In [15]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('', 0)
('\n', 1)
(' ', 2)
('!', 3)
('"', 4)
("'", 5)
('(', 6)
(')', 7)
(',', 8)
('--', 9)
('.', 10)
(':', 11)
(';', 12)
('?', 13)
('A', 14)
('Ah', 15)
('Among', 16)
('And', 17)
('Are', 18)
('Arrt', 19)
('As', 20)
('At', 21)
('Be', 22)
('Begin', 23)
('Burlington', 24)
('But', 25)
('By', 26)
('Carlo', 27)
('Chicago', 28)
('Claude', 29)
('Come', 30)
('Croft', 31)
('Destroyed', 32)
('Devonshire', 33)
('Don', 34)
('Dubarry', 35)
('Emperors', 36)
('Florence', 37)
('For', 38)
('Gallery', 39)
('Gideon', 40)
('Gisburn', 41)
('Gisburns', 42)
('Grafton', 43)
('Greek', 44)
('Grindle', 45)
('Grindles', 46)
('HAD', 47)
('Had', 48)
('Hang', 49)
('Has', 50)


- Below, we illustrate the tokenization of a short sample text using a small vocabulary:

<img src="figures/05.png" width="800px">

- Let's now put it all together into a tokenizer class. This is much more easier with a function

In [16]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- The `encode` function turns text into token IDs
- The `decode` function turns token IDs back into text

<img src="figures/06.png" width="800px">

- We can use the tokenizer to encode (that is, tokenize) texts into integers
- These integers can then be embedded (later) as input of/for the LLM

In [17]:
#Using an encode example with the function
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[4, 59, 5, 853, 991, 605, 536, 749, 8, 1129, 599, 8, 4, 70, 10, 41, 854, 1111, 757, 796, 10]


- We can decode the integers back into text

In [18]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [19]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'


# 2.3 BytePair encoding (using a pre trained tokenizer)

- GPT-2 used BytePair encoding (BPE) as its tokenizer
- it allows the model to break down words that aren't in its predefined vocabulary into smaller subword units or even individual characters, enabling it to handle out-of-vocabulary words
- For instance, if GPT-2's vocabulary doesn't have the word "unfamiliarword," it might tokenize it as ["unfam", "iliar", "word"] or some other subword breakdown, depending on its trained BPE merges
- The original BPE tokenizer can be found here: [https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- In this lecture, we are using the BPE tokenizer from OpenAI's open-source [tiktoken](https://github.com/openai/tiktoken) library, which implements its core algorithms in Rust to improve computational performance
- (Based on an analysis [here](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/02_bonus_bytepair-encoder/compare-bpe-tiktoken.ipynb), I found that `tiktoken` is approx. 3x faster than the original tokenizer and 6x faster than an equivalent tokenizer in Hugging Face)

In [ ]:
# pip install tiktoken

In [20]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.8.0


In [21]:
tokenizer = tiktoken.get_encoding("gpt2")

In [25]:
# <|endoftext|> is a token used to indicate that a book/ website in a dataset ends. And that a new book / website will start

In [22]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [26]:
tokenizer.encode("gwegwegg") # it recognizes unknown words

[70, 732, 70, 732, 1130]

In [23]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


- BPE tokenizers break down unknown words into subwords and individual characters:

<img src="figures/07.png" width="700px">

In [24]:
tokenizer.encode("Akwirw ier", allowed_special={"<|endoftext|>"})

[33901, 86, 343, 86, 220, 959]



# 2.4 Data sampling with a sliding window

- Above, we took care of the tokenization (converting text into word tokens represented as token ID numbers)
- Now, let's talk about how we create the data loading for LLMs
- We train LLMs to generate one word at a time, so we want to prepare the training data accordingly where the next word in a sequence represents the target to predict

<img src="figures/08.png" width="800px">

- For this, we use a sliding window approach, changing the position by +1:

<img src="figures/09.png" width="900px">

- Note that in practice it's best to set the stride equal to the context length so that we don't have overlaps between the inputs (the targets are still shifted by +1 always). However, this is only used nowadays most for research. Llama and chatgpt 4 use stride of 1.

<img src="figures/10.png" width="800px">

In the following code we have the input, and the target shifted by one position. And we don't have overlap so we have the stride with the same lenght of the input

In [30]:
from supplementary import create_dataloader_v1

#in this case we have 4 per row. but in gpt 4 we have something like 1024 per row
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
